# Mapping Survey Respondents at the County Level

In this jupyter notebook I use python libraries to complete a data cleaning project so that I can map the current number of responses in a dataset in QGIS 3.1.

In [1]:
# Import the relevant packages
import pandas as pd

# Import the sample data
df_zip = pd.read_csv('sample_data.csv')

Depending on the data that you have, I am strictly looking to map the number of responses in each county. This means I only want a column of zip codes and a column of counts. I suggest that if you have a large dataframe, to make a smaller one containing only these two variables.

In [2]:
# Sum the number of responses per zip code
df_zip = df_zip['zip'].value_counts().to_frame()

# Move the columns out of the index
df_zip = df_zip.reset_index()

# Name the columns
df_zip.columns = ['zip','count']

print(df_zip.head(3))

     zip  count
0  99403      3
1  99133      3
2  99032      3


Now that we have the dataframe that we want, we will need to import another csv that contains the zip code to state-county FIPS codes that we want to lookup. The reason we do this is that the US Census shapefiles attributes include a state FIPS and a county FIPS. In QGIS I will create a new attribute that is the combination for the two.

For example, King County (033) in Washington (53) is (53) + (033) = 53033. So let's begin first by importing the lookup table.

In [3]:
# Import the lookup csv file
df_lookup = pd.read_csv('lookup.csv')

# Rename to lowercase (personal preference)
df_lookup.columns = ['zip', 'county', 'state', 'stcountyfips', 'classfp'] 

print(df_lookup.head(3))

     zip          county state  stcountyfips classfp
0  36003  Autauga County    AL          1001      H1
1  36006  Autauga County    AL          1001      H1
2  36067  Autauga County    AL          1001      H1


Now that I have the two dataframes, all I need to do is merge the two based on some like condition. In this case, if the zip code from the survey responses dataframe (df_zip) matches a row in the lookup table (df_lookup), then I want to grab that row and add it to the survey responses dataframe.

In [4]:
# Merge on an equal column name across both dataframes
df_zip = df_zip.merge(df_lookup, on='zip')

print(df_zip.head(3))

     zip  count           county state  stcountyfips classfp
0  99403      3    Asotin County    WA         53003      H1
1  99403      3  Garfield County    WA         53023      H1
2  99403      3   Whitman County    WA         53075      H1


This appended all the data from the lookup table to the zipcode data. We now have a csv file that we can export that we can preform a join on in QGIS. We will join on the stcountyfips column which we will also have to make in QGIS for the census shapefile attributes. The graduated symoblogy will simply be based on the count. The problem is, in the above form, multiple enteries exist for state-county combinations since there are more than one zip code for each county. If we import this into GIS, it won't know how to handle it. It needs a single observation for each state-county combination. To do this, we simply aggregate the number of counts by state-county combination.

In [5]:
# Group by the state-county and aggregate the count column
df_zip = df_zip.groupby(['stcountyfips'], as_index = False).agg({'count' : 'sum'}) 

print(df_zip.head(3))

   stcountyfips  count
0         53001     22
1         53003      5
2         53005     16


This is exactly what we need for GIS analysis. To test, lets see what the number of responses are for King County.

In [6]:
print("There are " + df_zip.loc[df_zip['stcountyfips'] == 53033, 'count'].values[0].astype(str) + " responses in King County, WA")

There are 134 responses in King County, WA


In [ ]:
# Export the data into a csv that can be loaded into GIS
df_zip.to_csv('county_data.csv')

# Result

The following map is the outcome after using QGIS 3.1 to map the data. I loaded in the Census shapefiles for counties and states and in the counties attribute table, used the field calculator to create a column that is the state-county FIPS just like I made previously.

The resulting map is giberish since the sample data was randomly generated, but can be found in the Github repository as sample_map.pdf.